# Flight Visualization

## 1. Data Prepocessing

In [0]:
# First dataset is downloaded from my google drive. 
# This is because the csv file contains 

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link='https://drive.google.com/open?id=11ZZbDWJaF9PGSXjgFywE9d4elb1TTJyM'
# link='https://drive.google.com/open?id=1JOY4krAga6HWEXfQP8ePkYxpS9fhUhH2'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Flight Data.csv')  
df = pd.read_csv('Flight Data.csv')

In [2]:
df.head(5)

AirSpace_Class From_City    To_City  Price Aircraft_Model Engine_Model
0              B    Sydney  Melbourne    180       A330-203   CF6-80E142
1              A    Sydney   Brisbane    170       A330-202   CF6-80E142
2              B    Sydney   Canberra    120       B737-3B7    CFM56-3B1
3              B  Canberra     Sydney    120       B737-476     CFM-56-3
4              A    Sydney  Newcastle     90       A320-232     V2527-5A

In [3]:
#2.3.1 check how many unique categories each attributes has
for col_name in df.columns:
    if df[col_name].dtypes == 'object':
        unique_cat = len(df[col_name].unique())
        print ("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'AirSpace_Class' has 5 unique categories
Feature 'From_City' has 20 unique categories
Feature 'To_City' has 20 unique categories
Feature 'Aircraft_Model' has 7 unique categories
Feature 'Engine_Model' has 6 unique categories


In [4]:
df['From_City'].value_counts().sort_values(ascending=False)

Melbourne       6
Brisbane        5
Sydney          5
Adelaide        5
Perth           4
Darwin          4
BrokenHill      2
Canberra        2
Launceston      2
MtIsa           2
Hobart          2
Newcastle       1
AliceSprings    1
Cairns          1
Albany          1
Rockhampton     1
PtAugusta       1
Broome          1
Bendigo         1
Kalgoorlie      1
Name: From_City, dtype: int64

In [5]:
#calculate frequencies
to_ct=df['To_City'].value_counts().sort_values(ascending=False)
from_ct=df['From_City'].value_counts().sort_values(ascending=False)
res=pd.concat([to_ct,from_ct],axis=1)
print(res)
print(res.shape)

              To_City  From_City
Adelaide            5          5
Albany              1          1
AliceSprings        1          1
Bendigo             1          1
Brisbane            5          5
BrokenHill          2          2
Broome              1          1
Cairns              1          1
Canberra            2          2
Darwin              4          4
Hobart              2          2
Kalgoorlie          1          1
Launceston          2          2
Melbourne           6          6
MtIsa               2          2
Newcastle           1          1
Perth               4          4
PtAugusta           1          1
Rockhampton         1          1
Sydney              5          5
(20, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# convert 'locations' into longitude and latitude values
longitude_dict = {'Adelaide':138.6007, 'Albany': 117.8837, 'AliceSprings': 133.8807, 'Bendigo': 144.2794, 'Brisbane': 153.0251,
                  'BrokenHill': 141.4539,'Broome':122.2304, 'Cairns': 145.7781, 'Canberra':149.1300,'Darwin':130.8456,
                  'Hobart':147.3272,'Kalgoorlie':121.4660,'Launceston':147.1441,'Melbourne':144.9631,'MtIsa':139.4927,
                  'Newcastle':151.7817,'Perth':115.8605,'PtAugusta':137.7894,'Rockhampton':150.5100, 'Sydney':151.2093}
                  
                  
                  
                  
latitude_dict = {'Adelaide':-34.9285, 'Albany': -35.0269, 'AliceSprings': -23.6980, 'Bendigo': -36.7570, 'Brisbane': -27.4698,
                 'BrokenHill': -31.9539, 'Broome':-17.9644,'Cairns': -16.9186,'Canberra':-35.2809,'Darwin':-12.4634,
                 'Hobart':-42.8821,'Kalgoorlie':-30.7490,'Launceston':-41.4332,'Melbourne':-37.8136,'MtIsa':-20.7256,
                 'Newcastle':-32.9283,'Perth':-31.9505,'PtAugusta':-32.4952,'Rockhampton':-23.3791,'Sydney':-33.8688}




# To_City_lon=df.To_City.replace(longitude_dict,inplace=False)
# print(type(To_City_lon))
# pd.concat([df, pd.DataFrame(To_City_lon)], axis=1)
df['to_lon'] = df['To_City'].map(longitude_dict)
df['to_lat'] = df['To_City'].map(latitude_dict)
df['from_lon'] = df['From_City'].map(longitude_dict)
df['from_lat'] = df['From_City'].map(latitude_dict)
df.head()

AirSpace_Class From_City    To_City  ...   to_lat  from_lon from_lat
0              B    Sydney  Melbourne  ... -37.8136  151.2093 -33.8688
1              A    Sydney   Brisbane  ... -27.4698  151.2093 -33.8688
2              B    Sydney   Canberra  ... -35.2809  151.2093 -33.8688
3              B  Canberra     Sydney  ... -33.8688  149.1300 -35.2809
4              A    Sydney  Newcastle  ... -32.9283  151.2093 -33.8688

[5 rows x 10 columns]

## 2. Drawing Map

In [0]:
import folium
# folium_map = folium.Map(location=[40.738, -73.98],
#                         zoom_start=13,
#                         tiles="CartoDB dark_matter")
# marker = folium.CircleMarker(location=[40.738, -73.98])
# marker.add_to(folium_map)

In [0]:
for from_city in df['From_City'].unique():
  to_ct_df=df[df['From_City'] == from_city]
  mark_text=('From '+from_city+':\n')
  for i in range (len(to_ct_df['Price'])):
    price=str(to_ct_df['Price'].iloc[i])
    to_ct=to_ct_df['To_City'].iloc[i]
    mark_text+=('To '+to_ct+': '+price+'$\n') 
#   for i in range (len(to_ct_list)):
#     print(to_ct_list.loc[i,:])

#   marker=folium.Marker(locationlist[point], popup=df['To_City'][point]).add_to(map)
#   marker=folium.Marker(locationlist[point], popup=mark_text).add_to(map)

In [0]:
# locations = df[['to_lat', 'to_lon']]
# locationlist = locations.values.tolist()
# len(locationlist)

In [16]:
# draw map
_Widthfactor=3

ASC_dist={'A':'#ff0000','B':'#ff8c00','C':'#fff200','D':'#11ff00','E':'#339cb3'}

from folium import plugins
from folium.map import Popup
line_map = folium.Map(width='100%',height='100%',location=[-25.27, 133.77], zoom_start=4)


# for point in range(0, len(locationlist)):
  
for from_city in df['From_City'].unique():
  to_ct_df=df[df['From_City'] == from_city]
  mark_text=''  
  mark_tooltip=('From City: '+from_city)
  for i in range (len(to_ct_df['Price'])):
    price=str(to_ct_df['Price'].iloc[i])
    to_ct=to_ct_df['To_City'].iloc[i]
    mark_text+=('To '+to_ct+': '+price+'$. <br><br> ')
    

  locations=[to_ct_df['from_lat'].iloc[0],to_ct_df['from_lon'].iloc[0]]    

#   marker=folium.Marker(locationlist[point], popup=df['To_City'][point]).add_to(map)
  marker=folium.Marker(locations, tooltip =mark_tooltip,popup=mark_text).add_to(line_map)    
  
#   marker=folium.Marker(locationlist[point], popup=df['To_City'][point]).add_to(map)
  
  ic1 = plugins.BeautifyIcon(icon='plane', border_color='#7133b3', text_color='#b3334f',spin=True,icon_shape='marker').add_to(marker) 
  
points=[]
des = df[['to_lat', 'to_lon']]
starts = df[['from_lat', 'from_lon']]

plugins.Draw(
     export=True,
     filename='my_data.geojson',
     position='topleft',
     draw_options={'polyline': {'allowIntersection': False}},
     edit_options={'poly': {'allowIntersection': False}}
).add_to(line_map)

for i in range(len(des)) :
  points.append([[starts.iloc[:,0][i],starts.iloc[:,1][i]],[des.iloc[:,0][i],des.iloc[:,1][i]]])
  

for i in range (len(points)):
    
  # add an intermediate point
  
  x1=points[i][0][0]
  y1=points[i][0][1]
  x2=points[i][1][0]
  y2=points[i][1][1]
  median=[(x1+x2)/2-(x1-x2)/15,(y1+y2)/2-(y2-y1)/15]
  loca=[points[i][0],median,points[i][1]]
  
  
  # width of the line shows the price of the flight
  width=df['Price'][i]/df['Price'].max()*_Widthfactor
  
  # define color for AirSpace_Class
  
  
  route=folium.PolyLine(loca, weight=width+2, color=ASC_dist[df['AirSpace_Class'][i]], opacity=0.8).add_to(line_map)
#   plugins.PolyLineOffset(points[i], color="#080", opacity=1, offset=10).add_to(line_map)


  
  attr = {'fill': 'red'}
  # 飞机图例
  aircraft = {'font-weight': 'bold', 'font-size': '30'}
  
  
  plane_description='Price: '+str(df['Price'][i])+'$ <br>Air Space Class: '+df['AirSpace_Class'][i]+'<br>Engine Model: '+df['Engine_Model'][i]+'<br>Aircraft Model: '+df['Aircraft_Model'][i]
#   print(type(plane_description))
  
  atp=folium.plugins.AntPath(
    locations=loca,
    tooltip =plane_description,
    weight=width,
    dash_array=[20, 30]
    ).add_to(line_map)
  line_map.fit_bounds(line_map.get_bounds())
  
  

#   plugins.BeautifyIcon(icon='plane').add_to(atp)
  plane=plugins.PolyLineTextPath(
      atp,
      # '\u25BA', # 图例样式
      '\u2708',
      repeat=False,
      center=True,
      attributes=aircraft
  ).add_to(atp)

  
  
minimap = plugins.MiniMap()
line_map.add_child(minimap)
line_map

## 3. Save map

In [0]:
line_map.save('Flight_Visualiztion.html')

In [12]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'Flight_Visualiztion.html'})
uploaded.SetContentFile('Flight_Visualiztion.html')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 10tqBNf-skS1ykqCoaIbIVRVM1LpqrIHG
